Grab relevant information for bus stops for a specific route, join the data together.

In [1]:
import requests
import pandas as pd
import glob
import json
import time

with open('config.json', 'r') as f:
    config = json.load(f)

SWIFTLY_API_KEY = config['DEFAULT']['SWIFTLY_API_KEY']
# mssql_login = config['DEFAULT']['MSSQL_AMIGO_ADMIN']

# DEBUG = True

In [2]:
def pull_ridership_by_stop(line_number):

    # allFiles = glob.glob('.' + "/gps_*.csv")
    frame = pd.DataFrame()
    frames = []
    for file_ in ['Ridership/WEEKDAY.XLSX','Ridership/LRTWEEKDAY.XLSX']:
        df = pd.read_excel(file_, header=0)
        df['DAY']='RS_WKDY'
        #     df['DAY']='Weekday'
        frames.append(df)
    # for file_ in ['Ridership/SATURDAY.XLSX','Ridership/LRTSATURDAY.XLSX']:
    #     df = pd.read_excel(file_, header=0)
    #     df['DAY']='RS_SAT'
    #     frames.append(df)
    # for file_ in ['Ridership/SUNDAY.XLSX','Ridership/LRTSUNDAY.XLSX']:
    #     df = pd.read_excel(file_, header=0)
    #     df['DAY']='RS_SUN'
    #     frames.append(df)

    df = pd.concat(frames)
    df = df[~df['ROUTE_NUMBER'].isin([911,912,913,914])]

    df = df.query("DAY=='RS_WKDY'&ROUTE_NUMBER=='%d'" % line_number)

    #     pd.pivot_table(df.reset_index(), index=["STOP_ID"],values=["BOARD_ALL",'ALIGHT_ALL']).head()

    rid_line = pd.pivot_table(df.reset_index(), index=["STOP_ID","DIRECTION_NAME","TIME_PERIOD"],values=["SORT_ORDER","BOARD_ALL",'ALIGHT_ALL','LOAD_ALL','AVG_SERVICED','TIME_PERIOD_SORT']).reset_index().sort_values(by=['DIRECTION_NAME','TIME_PERIOD_SORT','SORT_ORDER'])
    rid_line['ALIGHT_ALL']= rid_line['ALIGHT_ALL'].round(2)
    rid_line['AVG_SERVICED'] = rid_line['AVG_SERVICED'].round(2)
    rid_line['BOARD_ALL'] = rid_line['BOARD_ALL'].round(2)
    rid_line['LOAD_ALL'] = rid_line['LOAD_ALL'].round(2)
    return rid_line

In [3]:
def pull_early_late_by_stop(line_number,SWIFTLY_API_KEY, dateRange, timeRange):
    '''http://dashboard.goswift.ly/vta/api-guide/docs/otp'''
#     line_number = '22'
#     dateRange = '10012017-10302017'
#     timeRange = '0500-0900'
    line_table = pd.read_csv('line_table.csv')
    line_table.rename(columns={"DirNum":"direction_id","DirectionName":"DIRECTION_NAME"},inplace=True)
    line_table['direction_id'] = line_table['direction_id'].astype(str)
    headers = {'Authorization': SWIFTLY_API_KEY}
    payload = {'agency': 'vta', 'route': line_number, 'dateRange': dateRange,'timeRange': timeRange, 'onlyScheduleAdherenceStops':'True'}
    # payload = {'agency': 'vta', 'dateRange': '10292017-10302017'}
    url = 'https://api.goswift.ly/otp/by-stop'
    r = requests.get(url, headers=headers, params=payload)
    try:
        swiftly_df = pd.DataFrame(r.json()['data'])
        swiftly_df.rename(columns={"stop_id":"STOP_ID"},inplace=True)
        swiftly_df = pd.merge(swiftly_df,line_table.query('lineabbr==%s'%line_number)[['direction_id','DIRECTION_NAME']])
        swiftly_df['STOP_ID'] = swiftly_df['STOP_ID'].astype(int)
        return swiftly_df
    except KeyError:
        print(r.json())

In [4]:
rid_by_stop_df = pull_ridership_by_stop(22)
rid_by_stop_df.head()

,STOP_ID,DIRECTION_NAME,TIME_PERIOD,ALIGHT_ALL,AVG_SERVICED,BOARD_ALL,LOAD_ALL,SORT_ORDER,TIME_PERIOD_SORT
186,328,EAST,AM Early,0.49,0.99,43.87,43.99,10,1
192,329,EAST,AM Early,0.00,0.19,0.64,44.62,20,1
198,330,EAST,AM Early,0.00,0.05,0.16,44.79,30,1
204,331,EAST,AM Early,0.00,0.01,0.04,44.83,40,1
210,332,EAST,AM Early,0.00,0.00,0.00,44.83,50,1


In [ ]:
# Midday - 9 to 2:30
# PM Peak - 2:30 to 6:30
# PM Late - 6:30 to 9:59
# PM Nite - 10pm to 12pm
# PM Nite - 12am to 3am

In [5]:
frames = []

df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0300-0459')
df['TIME_PERIOD'] = 'AM Early'
frames.append(df)
time.sleep(10)
df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0500-0829')
df['TIME_PERIOD'] = 'AM Peak'
frames.append(df)
time.sleep(10)
df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0900-1429')
df['TIME_PERIOD'] = 'Midday'
frames.append(df)
time.sleep(10)
df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '1430-1829')
df['TIME_PERIOD'] = 'PM Peak'
frames.append(df)
time.sleep(10)
# df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '1830-2199')
# df['TIME_PERIOD'] = 'PM Late'
# frames.append(df)
# time.sleep(10)
# df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '2200-2400')
# df['TIME_PERIOD'] = 'PM Nite'
# frames.append(df)
# time.sleep(10)
# df = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0000-0259')
# df['TIME_PERIOD'] = 'PM Nite'
# frames.append(df)

# e_l['TIMEPOINT'] = True

# e_l = pull_early_late_by_stop('22',SWIFTLY_API_KEY, dateRange = '10012017-10302017', timeRange = '0700-0900')
# e_l['TIME_PERIOD'] = 'AM Peak'

In [6]:
times = pd.concat(frames)
times['TIMEPOINT'] = True

In [7]:
times

,direction_id,early,late,ontime,STOP_ID,stop_name,total,DIRECTION_NAME,TIME_PERIOD,TIMEPOINT
0,0,2,19,66,328,Palo Alto Caltrain Station,87,EAST,AM Early,True
1,0,6,17,67,335,El Camino & California,90,EAST,AM Early,True
2,0,3,14,59,347,El Camino & Showers,76,EAST,AM Early,True
3,0,4,11,62,355,El Camino & Castro,77,EAST,AM Early,True
4,0,10,8,60,364,El Camino & Hollenbeck,78,EAST,AM Early,True
5,0,7,8,57,370,El Camino & Wolfe,72,EAST,AM Early,True
6,0,15,5,51,378,El Camino & Kiely,71,EAST,AM Early,True
7,0,2,7,44,1,Santa Clara Caltrain Station,53,EAST,AM Early,True
8,0,6,15,38,400,The Alameda & Naglee,59,EAST,AM Early,True
9,0,2,16,50,412,Santa Clara & 1st,68,EAST,AM Early,True


In [8]:
rid_by_stop_df.head()

,STOP_ID,DIRECTION_NAME,TIME_PERIOD,ALIGHT_ALL,AVG_SERVICED,BOARD_ALL,LOAD_ALL,SORT_ORDER,TIME_PERIOD_SORT
186,328,EAST,AM Early,0.49,0.99,43.87,43.99,10,1
192,329,EAST,AM Early,0.00,0.19,0.64,44.62,20,1
198,330,EAST,AM Early,0.00,0.05,0.16,44.79,30,1
204,331,EAST,AM Early,0.00,0.01,0.04,44.83,40,1
210,332,EAST,AM Early,0.00,0.00,0.00,44.83,50,1


In [9]:
bus_df = pd.merge(rid_by_stop_df,times, how='outer', on=['STOP_ID','DIRECTION_NAME','TIME_PERIOD'])
bus_df.to_csv("bus_analysis.csv",index=False)
bus_df.head()

,STOP_ID,DIRECTION_NAME,TIME_PERIOD,ALIGHT_ALL,AVG_SERVICED,BOARD_ALL,LOAD_ALL,SORT_ORDER,TIME_PERIOD_SORT,direction_id,early,late,ontime,stop_name,total,TIMEPOINT
0,328,EAST,AM Early,0.49,0.99,43.87,43.99,10,1,0,2.0,19.0,66.0,Palo Alto Caltrain Station,87.0,True
1,329,EAST,AM Early,0.00,0.19,0.64,44.62,20,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,330,EAST,AM Early,0.00,0.05,0.16,44.79,30,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,331,EAST,AM Early,0.00,0.01,0.04,44.83,40,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,332,EAST,AM Early,0.00,0.00,0.00,44.83,50,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
bus_df.query('TIMEPOINT==True')

,STOP_ID,DIRECTION_NAME,TIME_PERIOD,ALIGHT_ALL,AVG_SERVICED,BOARD_ALL,LOAD_ALL,SORT_ORDER,TIME_PERIOD_SORT,direction_id,early,late,ontime,stop_name,total,TIMEPOINT
0,328,EAST,AM Early,0.49,0.99,43.87,43.99,10,1,0,2.0,19.0,66.0,Palo Alto Caltrain Station,87.0,True
7,335,EAST,AM Early,0.09,0.26,1.02,45.27,80,1,0,6.0,17.0,67.0,El Camino & California,90.0,True
19,347,EAST,AM Early,0.30,0.26,0.97,49.02,200,1,0,3.0,14.0,59.0,El Camino & Showers,76.0,True
27,355,EAST,AM Early,1.20,0.39,0.29,50.30,280,1,0,4.0,11.0,62.0,El Camino & Castro,77.0,True
36,364,EAST,AM Early,0.86,0.32,0.17,50.04,370,1,0,10.0,8.0,60.0,El Camino & Hollenbeck,78.0,True
42,370,EAST,AM Early,0.16,0.08,0.17,50.55,430,1,0,7.0,8.0,57.0,El Camino & Wolfe,72.0,True
50,378,EAST,AM Early,0.34,0.40,1.37,52.03,510,1,0,15.0,5.0,51.0,El Camino & Kiely,71.0,True
60,1,EAST,AM Early,4.69,0.77,2.13,52.34,610,1,0,2.0,7.0,44.0,Santa Clara Caltrain Station,53.0,True
68,400,EAST,AM Early,0.34,0.35,1.07,56.58,690,1,0,6.0,15.0,38.0,The Alameda & Naglee,59.0,True
79,412,EAST,AM Early,14.04,0.96,2.94,46.85,800,1,0,2.0,16.0,50.0,Santa Clara & 1st,68.0,True


In [ ]:
bus_df